Notebook para a divisão k-fold

In [77]:
import pandas as pd

In [78]:
import numpy as np

In [79]:
def read_file_as_df(file_name):
    import pandas as pd
    import csv

    import sys
    import pandas as pd

    maxInt = sys.maxsize

    while True:
        # decrease the maxInt value by factor 10 
        # as long as the OverflowError occurs.

        try:
            csv.field_size_limit(maxInt)
            break
        except OverflowError:
            maxInt = int(maxInt/10)

    file = []
    col = []

    with open(file_name) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=';')
        line_count = 0
        for row in csv_reader:
            if line_count==0:
                for r in row:
                    col.append(r)
                line_count+=1
            else:
                line = []
                for r in row:
                    line.append(r)
                file.append(line)
                line_count += 1

            
    df = pd.DataFrame(file, columns = col)
    return df


In [80]:
df = read_file_as_df(r'C:\Users\alici\Documents\tcc\v3\bases\b5\b5_post_concat.csv')

In [81]:
df.shape

(1082, 24)

In [82]:
df.dropna(axis=1, inplace=True)
df.drop_duplicates(inplace=True)

In [83]:
df.shape

(1082, 24)

In [84]:
df.columns

Index(['User_ID', 'inventory', 'exp', 'ActualGender', 'age', 'TI',
       'religiousity', 'course', 'extraversion', 'agreeableness',
       'conscientiousness', 'neuroticism', 'openness', 'assertiveness',
       'activity', 'altruism', 'compliance', 'order', 'selfdiscipline',
       'ansiety', 'depression', 'aesthetics', 'ideas', 'Text'],
      dtype='object')

In [85]:
df = df.drop(columns=['TI',
       'religiousity', 'course', 'extraversion', 'agreeableness',
       'conscientiousness', 'neuroticism', 'openness', 'assertiveness',
       'activity', 'altruism', 'compliance', 'order', 'selfdiscipline',
       'ansiety', 'depression', 'aesthetics', 'ideas', 'inventory', 'exp'])

In [86]:
df.columns

Index(['User_ID', 'ActualGender', 'age', 'Text'], dtype='object')

In [87]:
df.shape

(1082, 4)

In [88]:
df['ActualGender'].value_counts()

f     597
m     484
na      1
Name: ActualGender, dtype: int64

In [89]:
df.drop(df[df['ActualGender'] == 'na'].index, inplace = True)

In [90]:
df.shape

(1081, 4)

In [91]:
df['age'] = df['age'].replace('', np.nan)
df['Text'] = df['Text'].replace('', np.nan)

In [92]:
df[df['Text'].isna()].shape

(62, 4)

In [93]:
df = df.dropna(subset=['age', 'Text'])

In [94]:
df.shape

(753, 4)

In [95]:
def age_class(text):
    age = int(float(text))
    if age >=18 and age<=20:
        return 1
    elif age >=23 and age<=25:
        return 2
    elif age >= 28 and age<=61:
        return 3
    else:
        return 4

In [96]:
df["AgeClass"] = df["age"].apply(age_class)

In [97]:
df.drop(df[df['AgeClass'] == 4].index, inplace = True)

In [98]:
df.shape

(516, 5)

In [99]:
def gender_class(text):
    if(text == "f"):
        return 1
    else:
        return 0

In [100]:
df["GenderClass"] = df["ActualGender"].apply(gender_class)

In [101]:
df_fem = df[df["GenderClass"]==1]
df_fem["AgeClass"].value_counts()

1    116
2     94
3     78
Name: AgeClass, dtype: int64

In [102]:
df_masc = df[df["GenderClass"]==0]
df_masc["AgeClass"].value_counts()

2    95
3    67
1    66
Name: AgeClass, dtype: int64

In [103]:
df.dropna(axis=1, inplace=True)
df.dropna(axis=1, inplace=True)

df.drop_duplicates(inplace=True)
df.drop_duplicates(inplace=True)

In [104]:
df.shape

(516, 6)

In [105]:
def age_and_gender_class(df):
    ageGender = []
    for _, line in df.iterrows():
        if line["AgeClass"]==1 and line["GenderClass"]==1:
            ageGender.append(1)
        elif line["AgeClass"]==1 and line["GenderClass"]==0:
            ageGender.append(2)
        elif line["AgeClass"]==2 and line["GenderClass"]==1:
            ageGender.append(3)
        elif line["AgeClass"]==2 and line["GenderClass"]==0:
            ageGender.append(4)
        elif line["AgeClass"]==3 and line["GenderClass"]==1:
            ageGender.append(5)
        elif line["AgeClass"]==3 and line["GenderClass"]==0:
            ageGender.append(6)
    df["AgeGender"] = ageGender

In [106]:
age_and_gender_class(df)

In [107]:
df.reset_index(drop=True, inplace=True)

In [108]:
from sklearn.model_selection import StratifiedKFold

In [109]:
kf_strat = StratifiedKFold(n_splits=10, shuffle=True)

In [110]:
x = df
y = df["AgeGender"]

In [111]:
def get_splits_strat(df, samples):
     kfold_list = []
     for train_index, test_index in kf_strat.split(df, samples):
          print(len(train_index), len(test_index))
          dftrain = df.iloc[train_index]
          dftest = df.iloc[test_index]
          kfold_list.append([dftrain, dftest])
     return kfold_list

In [112]:
kfold_list = get_splits_strat(x, y)

464 52
464 52
464 52
464 52
464 52
464 52
465 51
465 51
465 51
465 51


### Salvar em arquivo

In [113]:
def create_split_csv(l, path):
    import os
    import shutil

    root_folder = path
    folder_name = "k_"
    file_name1 = "train"
    file_name2 = "test"
    k = 1

    for training, test in l:
        path = os.path.join(root_folder, (folder_name+str(k)))
        if(os.path.exists(path)):
            shutil.rmtree(path)
        os.mkdir(path)

        training.to_csv((path+"/"+file_name1+".csv"), index = False, header=True)
        test.to_csv((path+"/"+file_name2+".csv"), index = False, header=True)

        k+=1

In [114]:
create_split_csv(kfold_list, r"C:\Users\alici\Documents\tcc\v3\particoes\b5")